In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [2]:
itVotesRows = sc.textFile("../Data/Italian_Stack_Exchange/italianVotes.csv")
itVotesSplit = itVotesRows.map(lambda x : x.encode('utf8').split("~"))

In [3]:
itVotesSplit.take(1)

[['2657', '135', '2', '2013-11-22 00:00:00.0']]

In [4]:
from pyspark.sql import Row
from datetime import datetime

def toTimeSafe(inval):
  try:
    return datetime.strptime(inval, "%Y-%m-%d %H:%M:%S.%f")
  except ValueError:
    return None

In [5]:
def stringToVote(r):
  return Row(
    long(r[0]),
    long(r[1]),
    int(r[2]),
    toTimeSafe(r[3])
    )

In [6]:
from pyspark.sql.types import *
voteSchema = StructType([
    StructField("id", LongType(), False),
    StructField("postId", LongType(), False),
    StructField("voteTypeId", IntegerType(), False),
    StructField("creationTime", TimestampType(), False),
    ])

In [7]:
rowRDD = itVotesSplit.map(lambda x: stringToVote(x))

In [8]:
itVotesDFStruct = sqlContext.createDataFrame(rowRDD, voteSchema)

In [9]:
itVotesDFStruct.printSchema()

root
 |-- id: long (nullable = false)
 |-- postId: long (nullable = false)
 |-- voteTypeId: integer (nullable = false)
 |-- creationTime: timestamp (nullable = false)



In [10]:
import pyspark.sql.functions as func
itVotesDFStruct.filter('voteTypeId = 1').sort('creationTime').show()

+----+------+----------+-------------------+
|  id|postId|voteTypeId|       creationTime|
+----+------+----------+-------------------+
|  53|    12|         1|2013-11-05 00:00:00|
|  46|     2|         1|2013-11-05 00:00:00|
| 195|    36|         1|2013-11-06 00:00:00|
| 232|    55|         1|2013-11-06 00:00:00|
| 220|    30|         1|2013-11-06 00:00:00|
| 216|    52|         1|2013-11-06 00:00:00|
| 181|    45|         1|2013-11-06 00:00:00|
| 364|    77|         1|2013-11-07 00:00:00|
| 380|    84|         1|2013-11-07 00:00:00|
| 375|    83|         1|2013-11-07 00:00:00|
| 501|    88|         1|2013-11-08 00:00:00|
|1651|  1158|         1|2013-11-10 00:00:00|
|1672|   106|         1|2013-11-10 00:00:00|
|1781|  1170|         1|2013-11-11 00:00:00|
|1895|  1212|         1|2013-11-12 00:00:00|
|1932|  1235|         1|2013-11-12 00:00:00|
|1979|  1228|         1|2013-11-13 00:00:00|
|2011|  1234|         1|2013-11-13 00:00:00|
|1982|  1216|         1|2013-11-13 00:00:00|
|2053|    